In [1]:
import glob
import time

import pandas as pd

In [2]:
data_files = glob.glob("*/results/*.csv")

In [3]:
df_github_names_long = pd.concat(
    [ pd.read_csv(fp) for fp in data_files], 
    axis=0, 
    keys=data_files, 
    names=["source", "row"]
).reset_index("source").reset_index(drop=True)
df_github_names_long

,source,github_user_id,uu_user_id
0,github_search\results\ids_search_repos.csv,jonasmoons,NaN
1,github_search\results\ids_search_repos.csv,gglucass,NaN
2,github_search\results\ids_search_repos.csv,MarcelRobeer,NaN
3,github_search\results\ids_search_repos.csv,ekatrukha,NaN
4,github_search\results\ids_search_repos.csv,UtrechtUniversity,NaN
...,...,...,...
518,university_profile_pages\results\employees_git...,ruudvdent,RJvanderEnt
519,university_profile_pages\results\employees_git...,UtrechtRiversEstuaries,MGKleinhans
520,university_profile_pages\results\employees_git...,billjee,ASobhani
521,university_profile_pages\results\employees_git...,UU-Hydro,EHSutanudjaja


In [4]:
df_github_names_crosstab = pd.crosstab(df_github_names_long["github_user_id"], df_github_names_long["source"])
df_github_names_crosstab

source,github_search\results\ids_search_repos.csv,github_search\results\ids_search_users.csv,github_search\results\ids_topic_repos.csv,pure\results\ids_pure_users.csv,university_profile_pages\results\employees_github_from_cv_urls.csv,university_profile_pages\results\employees_github_from_links_urls.csv,university_profile_pages\results\employees_github_from_profile_urls.csv
github_user_id,,,,,,,
19giorgosts,0,1,0,0,0,0,0
4009509,0,1,0,0,0,0,0
73616e646572,0,0,0,0,0,1,0
AJueling,1,1,0,0,0,0,0
ARYENN,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
xavierpinho23,0,1,0,0,0,0,0
yaikohi,0,1,0,0,0,0,0
ydluo,0,0,0,1,0,0,0


## Basic analytics

In [32]:
df_github_names_long.drop_duplicates(["github_user_id", "source"])["github_user_id"].value_counts()

ekatrukha            3
UU-Hydro             3
kevin4998            3
UtrechtUniversity    3
RELabUU              2
                    ..
Zupami               1
Garanas              1
koenklomps           1
PieterKnops          1
M-thieu              1
Name: github_user_id, Length: 418, dtype: int64

## Create unique table of users and corresponding SOLIS-ID

In [8]:
df_users = df_github_names_long[["github_user_id", "uu_user_id"]].sort_values("uu_user_id").drop_duplicates("github_user_id").reset_index(drop=True)
df_users

,github_user_id,uu_user_id
0,ajinkyakadu125,AAKadu
1,msdslab,AGJvandeSchoot
2,alexandrosstergiou,AGStergiou
3,annawegmann,AMWegmann
4,amacanovic,AMacanovic
...,...,...
413,keesmulder,NaN
414,RianneSchouten,NaN
415,ydluo,NaN
416,marnixnaber,NaN


In [9]:
from ghapi.all import GhApi, pages
import os
from dotenv import load_dotenv

load_dotenv()
# if unauthorized API is used, rate limit is lower leading to a ban and waiting time needs to be increased
token = os.getenv('GITHUB_TOKEN') 
api = GhApi(token = token)
if(token is None):
    sleep = 6
else:
    sleep = 2

In [10]:
results_github_user_api = []

for x in df_users["github_user_id"]:
    try:
        user = dict(api.users.get_by_username(x))
        results_github_user_api.append(user)
    except Exception as e: 
        print("User %s encountered an error." % x)
        print(e)
    if(len(results_github_user_api) % 10 == 0):
        print("Processed %d users." % len(results_github_user_api))
    time.sleep(sleep)

User ajinkyakadu125 encountered an error.
HTTP Error 404: Not Found
Processed 0 users.
User msdslab encountered an error.
HTTP Error 404: Not Found
Processed 0 users.
User davidcortesortuno">https: encountered an error.
HTTP Error 404: Not Found
Processed 10 users.
Processed 20 users.
Processed 30 users.
User iv4xr-project>project's encountered an error.
HTTP Error 404: Not Found
Processed 40 users.
User vitality-data-center>here< encountered an error.
HTTP Error 404: Not Found
Processed 50 users.
Processed 60 users.
Processed 70 users.
Processed 80 users.
Processed 90 users.
Processed 100 users.
Processed 110 users.
Processed 120 users.
Processed 130 users.
Processed 140 users.
User UU-Bachelor-s-Thesis-Ruijs-Walstra encountered an error.
HTTP Error 404: Not Found
Processed 150 users.
Processed 160 users.
Processed 170 users.
Processed 180 users.
Processed 190 users.
Processed 200 users.
Processed 210 users.
Processed 220 users.
Processed 230 users.
Processed 240 users.
Processed 250 

In [11]:
df_github_user_api = pd.DataFrame(results_github_user_api)
df_github_user_api

,login,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,gists_url,...,email,hireable,bio,twitter_username,public_repos,public_gists,followers,following,created_at,updated_at
0,alexandrosstergiou,25504317,MDQ6VXNlcjI1NTA0MzE3,https://avatars.githubusercontent.com/u/255043...,,https://api.github.com/users/alexandrosstergiou,https://github.com/alexandrosstergiou,https://api.github.com/users/alexandrosstergio...,https://api.github.com/users/alexandrosstergio...,https://api.github.com/users/alexandrosstergio...,...,alexstergiou5@gmail.com,True,Computer Vision and Machine Learning Researcher,None,20,0,35,6,2017-02-02T11:23:32Z,2021-04-16T12:44:56Z
1,AnnaWegmann,22360217,MDQ6VXNlcjIyMzYwMjE3,https://avatars.githubusercontent.com/u/223602...,,https://api.github.com/users/AnnaWegmann,https://github.com/AnnaWegmann,https://api.github.com/users/AnnaWegmann/follo...,https://api.github.com/users/AnnaWegmann/follo...,https://api.github.com/users/AnnaWegmann/gists...,...,None,None,None,None,2,0,2,2,2016-09-22T02:20:41Z,2021-03-29T15:28:02Z
2,amacanovic,57662333,MDQ6VXNlcjU3NjYyMzMz,https://avatars.githubusercontent.com/u/576623...,,https://api.github.com/users/amacanovic,https://github.com/amacanovic,https://api.github.com/users/amacanovic/followers,https://api.github.com/users/amacanovic/follow...,https://api.github.com/users/amacanovic/gists{...,...,None,None,None,None,3,0,2,1,2019-11-12T09:29:13Z,2021-03-17T10:28:21Z
3,billjee,3830672,MDQ6VXNlcjM4MzA2NzI=,https://avatars.githubusercontent.com/u/383067...,,https://api.github.com/users/billjee,https://github.com/billjee,https://api.github.com/users/billjee/followers,https://api.github.com/users/billjee/following...,https://api.github.com/users/billjee/gists{/gi...,...,None,None,None,None,5,1,22,5,2013-03-11T10:22:49Z,2021-03-19T15:38:29Z
4,bricker0,1919662,MDQ6VXNlcjE5MTk2NjI=,https://avatars.githubusercontent.com/u/191966...,,https://api.github.com/users/bricker0,https://github.com/bricker0,https://api.github.com/users/bricker0/followers,https://api.github.com/users/bricker0/followin...,https://api.github.com/users/bricker0/gists{/g...,...,None,None,Copernicus Institute for Sustainable Development,None,20,2,17,73,2012-07-03T20:37:22Z,2021-03-10T08:55:49Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,keesmulder,7806295,MDQ6VXNlcjc4MDYyOTU=,https://avatars.githubusercontent.com/u/780629...,,https://api.github.com/users/keesmulder,https://github.com/keesmulder,https://api.github.com/users/keesmulder/followers,https://api.github.com/users/keesmulder/follow...,https://api.github.com/users/keesmulder/gists{...,...,keestimmulder@gmail.com,None,Data Scientist at EY VODW,None,19,0,15,22,2014-06-05T14:26:12Z,2021-03-05T21:53:26Z
407,RianneSchouten,22293115,MDQ6VXNlcjIyMjkzMTE1,https://avatars.githubusercontent.com/u/222931...,,https://api.github.com/users/RianneSchouten,https://github.com/RianneSchouten,https://api.github.com/users/RianneSchouten/fo...,https://api.github.com/users/RianneSchouten/fo...,https://api.github.com/users/RianneSchouten/gi...,...,None,None,Missing Data Specialist / Developer Data & Ana...,None,10,0,17,0,2016-09-19T10:28:49Z,2021-03-26T15:28:53Z
408,ydluo,13563303,MDQ6VXNlcjEzNTYzMzAz,https://avatars.githubusercontent.com/u/135633...,,https://api.github.com/users/ydluo,https://github.com/ydluo,https://api.github.com/users/ydluo/followers,https://api.github.com/users/ydluo/following{/...,https://api.github.com/users/ydluo/gists{/gist...,...,None,None,None,None,1,0,18,0,2015-07-30T00:37:13Z,2021-03-26T14:37:03Z
409,marnixnaber,31031471,MDQ6VXNlcjMxMDMxNDcx,https://avatars.githubusercontent.com/u/310314...,,https://api.github.com/users/marnixnaber,https://github.com/marnixnaber,https://api.github.com/users/marnixnaber/follo...,https://api.github.com/users/marnixnaber/follo...,https://api.github.com/users/marnixnaber/gists...,...,None,None,None,None,1,0,2,0,2017-08-15T08:03:55Z,2021-03-19T08:40:20Z


In [12]:
def is_student(user_bio):
    """Checks whether a GitHub user is a student. The bio of a user is parsed. 
    If it contains phd the user will not be marked as a student. 
    If the bio contains only the word student the user will be marked as a student. If

    Args:
        user_id (string): user id which is named as "login" from the GitHub Api 

    Returns:
        Boolean: Whether the user is a student or not
    """
    user_bio = str(user_bio).lower()
    if (user_bio != "nan"):
        # PhD students should be included
        mention_phd = "phd" in user_bio
        mention_student = "student" in user_bio
        return (not mention_phd and mention_student)
    else:
        # we can't be sure and therefore keep the user
        return False

In [16]:
df_users_enriched = df_users.merge(df_github_user_api, left_on="github_user_id", right_on="login", how="left")
del df_users_enriched["login"]

In [17]:
df_users_enriched['is_student'] = df_users_enriched['bio'].apply(is_student)
df_users_enriched['is_student'].value_counts()

False    313
True     108
Name: is_student, dtype: int64

In [18]:
df_users_enriched.to_csv("unique_users.csv", index=False)